In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%cd ..

/home/ubuntu/sustaingym


# Scripts for generating events and running environment

In [2]:
from datetime import datetime, timedelta
import os
import pytz

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_style("darkgrid")

from sustaingym.data.load_moer import load_monthly_moer
from sustaingym.envs.evcharging import EVChargingEnv, RealTraceGenerator, GMMsTraceGenerator
from sustaingym.algorithms.evcharging.base_algorithm import GreedyAlgorithm, RandomAlgorithm, RLAlgorithm, MPC


test_ranges = (
    ('2019-05-01', '2019-08-31'),
    ('2019-09-01', '2019-12-31'),
    ('2020-02-01', '2020-05-31'),
    ('2021-05-01', '2021-08-31'),
)


## Real trace generator

In [3]:
for site in ['caltech', 'jpl']:
    for test_range in test_ranges:
        print('testing: ', site, test_range)
        rtg = RealTraceGenerator(site, test_range)
        for _ in range(123):  # 4 months -> 123 days maximum
            _, _, num_plug_events = rtg.get_event_queue()
        print(rtg)

testing:  caltech ('2019-05-01', '2019-08-31')
RealTracesGenerator from the Caltech site from 2019-05-01 to 2019-08-31. Current day 2019-08-31. 
testing:  caltech ('2019-09-01', '2019-12-31')
RealTracesGenerator from the Caltech site from 2019-09-01 to 2019-12-31. Current day 2019-09-01. 
testing:  caltech ('2020-02-01', '2020-05-31')
RealTracesGenerator from the Caltech site from 2020-02-01 to 2020-05-31. Current day 2020-02-02. 
testing:  caltech ('2021-05-01', '2021-08-31')
RealTracesGenerator from the Caltech site from 2021-05-01 to 2021-08-31. Current day 2021-08-31. 
testing:  jpl ('2019-05-01', '2019-08-31')
RealTracesGenerator from the Jpl site from 2019-05-01 to 2019-08-31. Current day 2019-08-31. 
testing:  jpl ('2019-09-01', '2019-12-31')
RealTracesGenerator from the Jpl site from 2019-09-01 to 2019-12-31. Current day 2019-09-01. 
testing:  jpl ('2020-02-01', '2020-05-31')
RealTracesGenerator from the Jpl site from 2020-02-01 to 2020-05-31. Current day 2020-02-02. 
testing: 

## GMMs trace generator

In [4]:
for site in ['caltech', 'jpl']:
    for test_range in test_ranges:
        print('testing: ', site, test_range)
        gmmg = GMMsTraceGenerator(site, test_range)
        for _ in range(123):  # 4 months -> 123 days maximum
            _, _, num_plug_events = gmmg.get_event_queue()
        print(gmmg)

testing:  caltech ('2019-05-01', '2019-08-31')
GMMsTracesGenerator from the Caltech site from 2019-05-01 to 2019-08-31. Sampler is GMM with 30 components. 
testing:  caltech ('2019-09-01', '2019-12-31')
GMMsTracesGenerator from the Caltech site from 2019-09-01 to 2019-12-31. Sampler is GMM with 30 components. 
testing:  caltech ('2020-02-01', '2020-05-31')
GMMsTracesGenerator from the Caltech site from 2020-02-01 to 2020-05-31. Sampler is GMM with 30 components. 
testing:  caltech ('2021-05-01', '2021-08-31')
GMMsTracesGenerator from the Caltech site from 2021-05-01 to 2021-08-31. Sampler is GMM with 30 components. 
testing:  jpl ('2019-05-01', '2019-08-31')
GMMsTracesGenerator from the Jpl site from 2019-05-01 to 2019-08-31. Sampler is GMM with 30 components. 
testing:  jpl ('2019-09-01', '2019-12-31')
GMMsTracesGenerator from the Jpl site from 2019-09-01 to 2019-12-31. Sampler is GMM with 30 components. 
testing:  jpl ('2020-02-01', '2020-05-31')
GMMsTracesGenerator from the Jpl site

## Rewards from policies

In [5]:
policies = ['random', 'full', 'none', 'selective_full']

for policy in policies:
    rewards = []
    reward_comps = {'profit': 0, 'carbon_cost': 0, 'excess_charge': 0}
    gmmg = GMMsTraceGenerator('caltech', test_ranges[0])
    env = EVChargingEnv(gmmg, action_type='discrete', moer_forecast_steps=36)

    num_episodes = 5
    for _ in range(num_episodes):
        obs = env.reset()
        done = False
        tot_reward = 0
        while not done:
            if policy == 'random':
                action = np.random.randint(0, 5, size=(54,))
            elif policy == 'full':
                action = np.full((54,), 4)
            elif policy == 'none':
                action = np.zeros((54,))
            else:
                action = np.where(obs['demands'] > 0, 4, 0)
            obs, reward, done, info = env.step(action)

            tot_reward += reward
            for reward_comp in info['reward']:
                reward_comps[reward_comp] += info['reward'][reward_comp]
        rewards.append(tot_reward)
    print(f'{policy} rewards: ', rewards)  # rewards per num_episodes
    print('reward components:', reward_comps)  # total reward contribution over num_episodes


random rewards:  [-9.20538625756031, -16.095760394232656, -40.353458203385635, -32.150978309804614, -19.847337928997227]
reward components: {'profit': 39.14599951563122, 'carbon_cost': 144.0161539449463, 'excess_charge': 12.782766664665491}
full rewards:  [-88.93214742348485, -76.00004425021787, -64.92499589865847, -41.43152301273302, -80.06867687006691]
reward components: {'profit': 37.86346411370879, 'carbon_cost': 316.83008619075076, 'excess_charge': 72.39076537812032}
none rewards:  [0.0, 0.0, 0.0, 0.0, 0.0]
reward components: {'profit': 0.0, 'carbon_cost': 0.0, 'excess_charge': 0.0}
selective_full rewards:  [10.726320747729439, 2.7434102707532584, 1.3634530237333227, 1.9140559962278771, 10.532486064969884]
reward components: {'profit': 44.10525285538047, 'carbon_cost': 16.648518143200974, 'excess_charge': 0.1770086087657736}


## Check seed setting is done correctly

In [3]:
print('RealTraceGenerator')
for sequential in [False, True]:
    for random_seed in [None, 11]:
        print(f'sequential {sequential}, random_seed {random_seed}')
        for _ in range(3):
            gen = RealTraceGenerator('caltech', test_ranges[1], sequential=sequential, random_seed=random_seed)

            num_events = []
            for _ in range(7):
                _, _, num_event = gen.get_event_queue()
                num_events.append(num_event)
            print(num_events)

print('GMMsTraceGenerator')
for random_seed in [None, 11]:
    print(f'random_seed {random_seed}')
    for _ in range(3):
        gen = GMMsTraceGenerator('caltech', test_ranges[1], random_seed=random_seed)
        num_events = []
        for _ in range(7):
            _, _, num_event = gen.get_event_queue()
            num_events.append(num_event)
        print(num_events)

RealTraceGenerator
sequential False, random_seed None
[29, 10, 36, 26, 23, 27, 31]
[34, 32, 35, 34, 28, 28, 4]
[31, 31, 9, 4, 1, 6, 30]
sequential False, random_seed 11
[32, 6, 20, 33, 23, 33, 30]
[32, 6, 20, 33, 23, 33, 30]
[32, 6, 20, 33, 23, 33, 30]
sequential True, random_seed None
[5, 11, 31, 30, 35, 36, 10]
[5, 11, 31, 30, 35, 36, 10]
[5, 11, 31, 30, 35, 36, 10]
sequential True, random_seed 11
[5, 11, 31, 30, 35, 36, 10]
[5, 11, 31, 30, 35, 36, 10]
[5, 11, 31, 30, 35, 36, 10]
GMMsTraceGenerator
random_seed None
[36, 9, 6, 29, 7, 30, 38]
[30, 31, 31, 35, 4, 11, 33]
[7, 33, 1, 8, 6, 31, 7]
random_seed 11
[35, 10, 39, 38, 30, 19, 9]
[35, 10, 39, 38, 30, 19, 9]
[35, 10, 39, 38, 30, 19, 9]


## Timestep Greedy and MPC

### Discrete Case

In [3]:
ga = GreedyAlgorithm(project_action=True)
mpc1 = MPC(lookahead=1)
mpc2 = MPC(lookahead=2)
mpc6 = MPC(lookahead=6)
mpc12 = MPC(lookahead=12)
mpc36 = MPC(lookahead=36)

lbls = ['greedy', 'mpc1', 'mpc2', 'mpc6', 'mpc12', 'mpc36']
algs = [ga, mpc1, mpc2, mpc6, mpc12, mpc36]

for alg, lbl in zip(algs, lbls):
    gen = RealTraceGenerator('caltech', ('2019-06-03', '2019-06-09'), sequential=True)
    env = EVChargingEnv(gen)
    rewards, breakdown = alg.run(7, env)
    
    print(f'{lbl} average reward: ', np.mean(rewards))
    print(f'{lbl} rewards: ', rewards)
    print(f'{lbl} reward breakdown: ', breakdown)

Simulating days of charging


100%|██████████| 7/7 [01:09<00:00,  9.86s/it]


greedy average reward:  9.103818438610483
greedy rewards:  [14.608341284506709, 11.582776506811785, 9.76263303053472, 9.84730337240914, 13.405210615787231, 2.36242955768457, 2.1580347025392252]
greedy reward breakdown:  {'profit': 73.44116032093493, 'carbon_cost': 9.714153917328483, 'excess_charge': 0.0002773333333333338}
Simulating days of charging


100%|██████████| 7/7 [00:30<00:00,  4.38s/it]


mpc1 average reward:  9.065079344029224
mpc1 rewards:  [14.155571434432602, 11.589002950875702, 9.953536333828946, 9.83566177896273, 13.401318649880784, 2.36242955768457, 2.1580347025392252]
mpc1 reward breakdown:  {'profit': 73.01204143156116, 'carbon_cost': 9.556486023356896, 'excess_charge': 0.0}
Simulating days of charging


...

mpc6 average reward:  9.126605184897844
mpc6 rewards:  [14.331925323820421, 11.674602743427544, 10.092089126169672, 9.833578033262635, 13.425810982445912, 2.373028102297423, 2.1552019828613016]
mpc6 reward breakdown:  {'profit': 72.81922910877137, 'carbon_cost': 8.932854147820182, 'excess_charge': 0.0001386666666666669}
Simulating days of charging


100%|██████████| 7/7 [00:50<00:00,  7.20s/it]


mpc12 average reward:  9.066689075020056
mpc12 rewards:  [14.268435569049782, 11.68326724123394, 9.884189091147373, 9.698283316292457, 13.41624729033393, 2.381177799531442, 2.1352232175514714]
mpc12 reward breakdown:  {'profit': 71.83706533648571, 'carbon_cost': 8.37010314467914, 'excess_charge': 0.0001386666666666669}
Simulating days of charging


100%|██████████| 7/7 [01:29<00:00, 12.76s/it]

mpc36 average reward:  8.755236023859949
mpc36 rewards:  [14.206162908331459, 11.24889253578141, 8.719296893071467, 9.53886951557487, 13.33367517033448, 2.263419242305014, 1.976335901620949]
mpc36 reward breakdown:  {'profit': 68.33055741571344, 'carbon_cost': 7.0423799153607725, 'excess_charge': 0.0015253333333333351}


### Continuous Case

In [6]:
for alg, lbl in zip(algs, lbls):
    gen = RealTraceGenerator('caltech', ('2019-06-03', '2019-06-09'), sequential=True)
    env = EVChargingEnv(gen, action_type='continuous')
    rewards, breakdown = alg.run(7, env)

    print(f'{lbl} average reward: ', np.mean(rewards))
    print(f'{lbl} rewards: ', rewards)
    print(f'{lbl} reward breakdown: ', breakdown)

Simulating days of charging


100%|██████████| 7/7 [01:08<00:00,  9.84s/it]


greedy average reward:  9.15312305135341
greedy rewards:  [14.73946295170452, 11.580331177092566, 9.988362172844129, 9.843150197362618, 13.403252969331234, 2.3592671885995906, 2.1580347025392252]
greedy reward breakdown:  {'profit': 73.72686496579581, 'carbon_cost': 9.653668939655637, 'excess_charge': 0.0013346666666666687}
Simulating days of charging


100%|██████████| 7/7 [00:30<00:00,  4.41s/it]


mpc1 average reward:  9.074372292724918
mpc1 rewards:  [14.221004058892053, 11.582590121781422, 9.961209517105905, 9.83231499383925, 13.406185466316995, 2.3592671885995906, 2.1580347025392252]
mpc1 reward breakdown:  {'profit': 73.10548875144511, 'carbon_cost': 9.584744035704347, 'excess_charge': 0.0001386666666666669}
Simulating days of charging


...

mpc6 average reward:  9.132565649341212
mpc6 rewards:  [14.375190524270714, 11.671187575681968, 10.098768997422443, 9.828358337244667, 13.427159174181057, 2.3716885473568032, 2.1556063892308193]
mpc6 reward breakdown:  {'profit': 72.89555446728872, 'carbon_cost': 8.96667846007515, 'excess_charge': 0.0009164618255215675}
Simulating days of charging


100%|██████████| 7/7 [00:49<00:00,  7.02s/it]


mpc12 average reward:  9.079287577546324
mpc12 rewards:  [14.33771424234256, 11.671406267863418, 9.918192930069637, 9.690321309337019, 13.421648793684708, 2.379744160659251, 2.1359853388676804]
mpc12 reward breakdown:  {'profit': 71.97790507862565, 'carbon_cost': 8.420916129664683, 'excess_charge': 0.001975906137184119}
Simulating days of charging


100%|██████████| 7/7 [01:28<00:00, 12.64s/it]

mpc36 average reward:  8.792959617789156
mpc36 rewards:  [14.386969259443795, 11.296497433271572, 8.763924318459628, 9.530409927916946, 13.332347607533718, 2.263419242305014, 1.9771495355934363]
mpc36 reward breakdown:  {'profit': 68.60606645319396, 'carbon_cost': 7.050200858590292, 'excess_charge': 0.005148270079856753}
